In [0]:
%sql
show catalogs;

In [0]:
spark.catalog.listCatalogs()


In [0]:
%sql
show schemas in samples;

In [0]:
%sql show tables in samples.tpch

In [0]:
from pyspark.sql.types import StructType, StructField, StringType, BooleanType
# Defines a schema for a dataframe
schema = StructType([
    StructField("TableName", StringType(), True),
    StructField("database", StringType(), True),
    StructField("description", StringType(), True),
    StructField("tableType", StringType(), True),
    StructField("isTemporary", BooleanType(), True)
])
# loads the table list from catelog into variable
tables = spark.catalog.listTables("samples.tpch")
# load the tables into a dictionary defined by key value pairs below for eac row in tables  
tables_dict = [
    {
        "TableName": table.name,
        "database": table.database,
        "description": table.description,
        "tableType": table.tableType,
        "isTemporary": table.isTemporary
    }
    for table in tables
]
# Displays spark dataframe created from the dictionary in variable obove paired with Schema defined previously
display(
    spark.createDataFrame(
        tables_dict,
        schema
    )
)

In [0]:
%sql
describe table extended samples.tpch.lineitem;

In [0]:
%sql
list '/Volumes/workspace/default/get_started_upload_demo'

In [0]:
%sql
show volumes in clickstream_data.v01

In [0]:
%sql
LIST '/Volumes/clickstream_data/v01/raw';

In [0]:
%sql
LIST '/Volumes/clickstream_data/v01/raw/sales-csv';

In [0]:
%sql
LIST '/Volumes/clickstream_data/v01/raw/events-kafka';

In [0]:
%sql
select * from text. `/Volumes/workspace/default/get_started_upload_demo`
limit 10;

In [0]:
df = (spark
      .read
      .format("text")
      .load("/Volumes/workspace/default/get_started_upload_demo")
      .limit (10)
)
display(df)

In [0]:
%sql
select * from csv. `/Volumes/workspace/default/get_started_upload_demo`
limit 10;

In [0]:
df = (spark
      .read
      .format("csv")
      .load("/Volumes/workspace/default/get_started_upload_demo")
      .limit (10)
)
display(df)

In [0]:
%sql
select * from csv. `/Volumes/clickstream_data/v01/raw/sales-csv/`;

In [0]:
%sql
Create or replace table workspace.default.demo_customers as 
select * from read_files( '/Volumes/workspace/default/get_started_upload_demo',format=> 'csv',header => true);
    
select * from workspace.default.demo_customers
limit 10;

In [0]:
df = (spark
      .read
      .format("csv")
      .load("/Volumes/workspace/default/get_started_upload_demo")
      .limit (10)
)
(df
 .write
 .mode('overwrite')
 .saveAsTable("workspace.default.demo_customer_py")
)

customers_tbl =(spark
                .table("workspace.default.demo_customer_py")
                .limit (10)
                )
display(customers_tbl)
 

In [0]:
%sql
show tables in workspace.default;
    

In [0]:
%sql
Create or replace Table workspace.default.demo_sales as 
select *
from read_files('/Volumes/clickstream_data/v01/raw/sales-csv/',
format=>'csv',
delimiter=>'|',
header=>true
);
select * from workspace.default.demo_sales
limit 10;
select Count(*) from workspace.default.demo_sales


In [0]:
## Define a data frame df by loading file into it
df=(spark
    .read
    .format("csv")
    .option("header","true")
    .option("delimiter","|")
    .load("/Volumes/clickstream_data/v01/raw/sales-csv/")   
    )
## write the frame into a table    
(df
 .write
 .mode('overwrite')
 .saveAsTable("workspace.default.demo_sales_py")
)
## read table into a data frame
sales_tbl =(spark
            .table("workspace.default.demo_sales_py")
            .limit (10)
            )
## display the data frame            
display(sales_tbl)




In [0]:
%sql
select * from samples.nyctaxi.trips
limit 10;


In [0]:
%sql
Create or replace table workspace.default.demo_trips_clean as 

select 
-- datetimes 
tpep_pickup_datetime,
tpep_dropoff_datetime,
timestampdiff(minute,tpep_pickup_datetime,tpep_dropoff_datetime) as ride_minutes,
--convert to datetime
date(tpep_pickup_datetime) as pickup_date,
year(tpep_pickup_datetime) as Pickup_year,
hour(tpep_pickup_datetime) as pickup_hour,
dayofweek(tpep_pickup_datetime) as pickup_day_num,
date_format(tpep_pickup_datetime,'EEE') as pickup_day_name,
month(tpep_pickup_datetime) as pickup_month,

date(tpep_dropoff_datetime) as dropoff_date,
year(tpep_dropoff_datetime) as dropoff_year,
hour(tpep_dropoff_datetime) as dropoff_hour,
dayofweek(tpep_dropoff_datetime) as dropoff_day_num,
date_format(tpep_dropoff_datetime,'E') as dropoff_day_name,
MONTH(tpep_dropoff_datetime) as dropoff_month,
trip_distance,
fare_amount,
round(try_divide(fare_amount,ride_minutes),2) as cost_per_minute,
round(try_divide(fare_amount,trip_distance),2) as cost_per_mile,
pickup_zip,
dropoff_zip,
concat(pickup_zip,'-',dropoff_zip) as pickup_dropoff
From samples.nyctaxi.trips;

In [0]:
%sql
select pickup_day_name,
round(avg(cost_per_mile),2) as avg_cost_per_mile1
from workspace.default.demo_trips_clean
group by pickup_day_name
order by avg_cost_per_mile1 desc

In [0]:
%sql
select Pickup_date, count(*) as total_rides
from workspace.default.demo_trips_clean
group by Pickup_date
order by total_rides;

Databricks visualization. Run in Databricks to view.